# Introduction

### Problem Description

Data-driven approaches are now used in many fields from business to science. Since data storage and computational power has become cheap, machine learning has gained popularity. However, the majority of tools that can extract dependencies from data, are designed for prediction problem. In this notebook a problem of decision support simulation is considered and it is shown that even good predictive models can lead to wrong conclusions. This occurs under some conditions summarized by an umbrella term called endogeneity. In particular, accuracy of predictions does not guarantee causal relationships detection.

Suppose that situation is as follows. There is a freshly-hired manager that can assign treatment to items in order to increase target metric. Treatment is binary, i.e. for each item it is assigned or it is absent. Because treatment costs something, its assignment should be optimized - only some items should be treated. A historical dataset of items performance is given, but the manager does not know that previously treatment was assigned predominantely based on values of just one parameter. Moreover, this parameter is not included in the dataset. To make the situation more weird, an extra assumption can be introduced - now it is impossible to measure values of the omitted parameter not only for old items, but also for a new ones too. By the way, manager wants to create a system that predicts an item's target metric in case of treatment and in case of absence of treatment. If this system is deployed, the manager can compare these two cases and decide whether effect of treatment worths its costs.

If machine learning approach results in good prediction scores, chances are that the manager does not suspect that important variable is omitted (at least until some expenses are generated by wrong decisions). Hence, domain knowledge and data understanding are still required for modelling based on data. This is of particular importance when datasets contain values that are produced by someone's decisions, because there is no guarantee that future decisions will not change dramatically. On the flip side, if all factors that affect decisions are included into a dataset, i.e. there is selection on observables for treatment assignment, a powerful enough model is able to estimate treatment effect correctly.

Probably, sections of the notebook that illustrate ways to overcome lack of important unobservable variables, will be released after some time.

### References

To read more about causality in data analysis, it is possible to look at these papers:

1. *Angrist J, Pischke J-S. Mostly Harmless Econometrics. Princeton University Press, 2009.*

2. *Varian H. Big Data: New Tricks for Econometrics. Journal of Economic Perspectives, 28(2): 3–28, 2013*

### Software Requirements

This notebook does not use any packages beyond a list of those that are quite popular in scientific computing. Use `conda` or `pip` to install any of them.

# Preparations

### General

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Startup settings can not suppress a warning from XGBRegressor and so this is needed.
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from xgboost import XGBRegressor

In [2]:
np.random.seed(seed=361)

### Synthetic Dataset Generation

Let us generate an unobservable parameter and an indicator of treatment such that they are highly correlated.

In [3]:
unobservable = np.hstack((np.ones(1000), np.zeros(1000)))
treatment = np.hstack((np.ones(900), np.zeros(1000), np.ones(100)))

In [4]:
np.corrcoef(unobservable, treatment)

array([[ 1. ,  0.8],
       [ 0.8,  1. ]])

Now create historical dataset that is used for learning predictive model.

In [5]:
def synthesize_dataset(unobservable, treatment,
                       first=None, second=None,
                       true_weights=np.array([10, 1, 2, 3, 1]).T):
    """
    A helper function for repetitive
    pieces of code.
    
    Creates a dataset, where target depends on
    `unobservable`, but `unobservable` is not
    included as a feature.

    @type unobservable: numpy.ndarray
    @type treatment: numpy.ndarray
    @type first: numpy.ndarray
    @type second: numpy.ndarray
    @type true_weights: numpy.ndarray
    @return: numpy.ndarray, numpy.ndarray
    """

    try:
        assert unobservable.shape == treatment.shape
    except AssertionError:
        raise ValueError("Arrays are not aligned.")
    try:
        len(true_weights) == 5
    except AssertionError:
        raise ValueError("Not supported length of weights.")

    if first is None:
        first = np.random.normal(size=unobservable.shape[0])
    if second is None:
        second = np.random.normal(size=unobservable.shape[0])
    features = np.vstack((unobservable, treatment,
                          first, second, first * second)).T
    target = np.dot(features, true_weights)
    return features[:, 1:], target

In [6]:
learning_X, learning_y = synthesize_dataset(unobservable, treatment)

Now create two datasets for simulation where the only difference between them is that in the first one treatment is absent and in the second one treatment is assigned to all items.

In [7]:
unobservable = np.hstack((np.ones(250), np.zeros(250)))

In [8]:
no_treatment = np.zeros(500)
full_treatment = np.ones(500)

In [9]:
no_treatment_X, no_treatment_y = synthesize_dataset(unobservable, no_treatment)
full_treatment_X, full_treatment_y = synthesize_dataset(unobservable, full_treatment,
                                                        no_treatment_X[:, 1],
                                                        no_treatment_X[:, 2])

Look at the data that are used for simulation.

In [10]:
no_treatment_X[:5, :]

array([[ 0.        , -1.13413915, -1.23892813,  1.4051169 ],
       [ 0.        , -0.45949823, -0.13317826,  0.06119518],
       [ 0.        , -0.21266401, -0.20021329,  0.04257816],
       [ 0.        ,  1.05046456, -0.69384769, -0.72886241],
       [ 0.        , -0.48485098,  0.32247408, -0.15635187]])

In [11]:
full_treatment_X[:5, :]

array([[ 1.        , -1.13413915, -1.23892813,  1.4051169 ],
       [ 1.        , -0.45949823, -0.13317826,  0.06119518],
       [ 1.        , -0.21266401, -0.20021329,  0.04257816],
       [ 1.        ,  1.05046456, -0.69384769, -0.72886241],
       [ 1.        , -0.48485098,  0.32247408, -0.15635187]])

In [12]:
no_treatment_y[:5]

array([ 5.42005421,  8.74266393,  9.01661028,  9.29052366,  9.84136841])

In [13]:
full_treatment_y[:5]

array([  6.42005421,   9.74266393,  10.01661028,  10.29052366,  10.84136841])

# Perfect Model and Poor Simulation

In [14]:
# TODO: Prediction-vs-fact scatterplots.